<a href="https://colab.research.google.com/github/anochima/Simpoobusiness/blob/master/FaceClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt


from tensorflow.keras import layers, preprocessing, Sequential, losses
from tensorflow.keras.layers import Input, Dense

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Loading the Image data**

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train data
path = '/content/drive/MyDrive/real_and_fake_face/'

image_gen = ImageDataGenerator(validation_split=.2)

train_data = image_gen.flow_from_directory(directory=path, 
                                          target_size=(256,256),
                                          class_mode='binary', 
                                          subset = 'training',
                                          seed=42,
                                          batch_size=32)

test_data = image_gen.flow_from_directory(directory=path,
                                         class_mode='binary',
                                         target_size=(256,256),
                                         subset='validation',
                                         seed=42,
                                         batch_size=32)

Found 1686 images belonging to 2 classes.
Found 421 images belonging to 2 classes.


In [4]:
# check out the class names in our dataset
image_classes = list(train_data.class_indices.keys())
image_classes

['face', 'not_face']

## **Data Augumentation**

We will be using Keras Sequential model api to build a data augmentation model as a layer to feed to our neural network (Model).



In [5]:
data_augmentation = Sequential([
                                layers.RandomFlip('horizontal_and_vertical'),
                                layers.RandomZoom(0.2),
                                # layers.RandomCrop(0.2, 0.2),
                                layers.RandomRotation(0.2)
])

## **Transfer Learning Fine Tuning**
Adjusting the pretrained model weights

In [7]:
#  Let's use functional Api Modelling to create our model this time

# create our base model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# create our input layer
inputs = tf.keras.layers.Input(shape=(228,228,3))

# freeze the model weights
base_model.trainable = False

# Add our data Augmentation layer
x = data_augmentation(inputs)

# Normalize if model is Resnet50v2
# x = tf.keras.layers.experimental.preprocessing.Rescale(1./255)(inputs)

# otherwise:

x = base_model(inputs, training=False)
print(x.shape)

# AveragePool (Aggregrate all the most important feature information, reduce number of computations)

x = tf.keras.layers.GlobalAveragePooling2D(name='GlobalAveragePooling')(x)

# Create teh output layer

outputs = Dense(1, activation='sigmoid')(x)

# Model
functional_api_model = tf.keras.Model(inputs, outputs)

functional_api_model.summary()

(None, 7, 7, 1280)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 228, 228, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 GlobalAveragePooling (Globa  (None, 1280)             0         
 lAveragePooling2D)                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 1,281
Non-trainable params: 4,049,571
_________________________________________________________________


## **Initialize a callback**


In [10]:
import datetime
time_stamp = datetime.datetime.now().strftime('%Y_%m_%d_%H%M%S')

def learning_rate_scheduler():
  """
  A learning rate callback
  """
  
  lr = tf.keras.callbacks.LearningRateScheduler(lambda epochs: 3e-1*10 ** (epochs/20))
  return lr
  
def model_checkpoint_callback(experiment_name):
  checkpoint_path = 'Checkpoint/'+time_stamp+'_'+experiment_name.replace(' ','_').lower()
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                           verbose=1,
                                                           save_best_weights_only=True)
  print('ModelCheckpoint callback saved to: ', checkpoint_path)
  return checkpoint_callback

def tensorboard_callback(log_dir, experiment_name):
  directory = log_dir +'/' + experiment_name + '/'+ time_stamp+'_'+experiment_name.replace(' ','_').lower()

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=directory)
  print('Tensorboard callback saved to:', directory)
  return tensorboard_callback

## **Compile and fit our model**

In [14]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 12.7 MB/s 
     |████████████████████████████████| 144 kB 47.9 MB/s 
     |████████████████████████████████| 180 kB 48.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=51069df5e13bb84fb4cf1f609cfb4f36a0f2b1c3c541cec3a859b442308a5871
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [17]:
# login to wandb
! wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

# Initialize wandb
wandb.init(project="Face Classification", entity='acnice')

In [20]:
wandb.config = {
  "epochs": 10,
  "batch_size": 128
}

In [11]:
functional_api_model.compile(loss=losses.binary_crossentropy,
                             optimizer=tf.keras.optimizers.Adam(),
                             metrics=['accuracy'])

# fit our model
history1 = functional_api_model.fit(train_data, epochs=5, 
                         callbacks=[model_checkpoint_callback('Base Model'),
                                    tensorboard_callback('Base Model','Tensor Hub'), wandb], 
                         validation_data=test_data,
                         steps_per_epoch=len(train_data),
                          )

ModelCheckpoint callback saved to:  Checkpoint/2022_02_16_135714_base_model
Tensorboard callback saved to: Base Model/Tensor Hub/2022_02_16_135714_tensor_hub
Epoch 1/5
53/53 [==============================] - ETA: 0s - loss: 0.2701 - accuracy: 0.9241 
Epoch 1: saving model to Checkpoint/2022_02_16_135714_base_model
INFO:tensorflow:Assets written to: Checkpoint/2022_02_16_135714_base_model/assets
53/53 [==============================] - 990s 18s/step - loss: 0.2701 - accuracy: 0.9241 - val_loss: 0.1701 - val_accuracy: 0.9549
Epoch 2/5
53/53 [==============================] - ETA: 0s - loss: 0.0603 - accuracy: 0.9988
Epoch 2: saving model to Checkpoint/2022_02_16_135714_base_model
INFO:tensorflow:Assets written to: Checkpoint/2022_02_16_135714_base_model/assets
53/53 [==============================] - 69s 1s/step - loss: 0.0603 - accuracy: 0.9988 - val_loss: 0.1386 - val_accuracy: 0.9477
Epoch 3/5
53/53 [==============================] - ETA: 0s - loss: 0.0341 - accuracy: 0.9994
Epoch 3:

In [12]:
functional_api_model.evaluate(test_data)

14/14 [==============================] - 4s 264ms/step - loss: 0.1361 - accuracy: 0.9359


[0.1360691785812378, 0.9358670115470886]

In [13]:
# save our model
functional_api_model.save('FaceClassifier')

INFO:tensorflow:Assets written to: FaceClassifier/assets


## **Plot Model Curves**

In [ ]:
def plot_lr(history):
  
  epochs = history.history['loss']
  lr = 3e-1*10**(tf.range(len(epochs))/20)
  
  plt.plot(lr, epochs)
  plt.xlabel('Learning Rate')
  plt.ylabel('Loss')
  plt.title('Learning Rate')